In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
! pip install ktrain

     |████████████████████████████████| 25.3 MB 100 kB/s 
     |████████████████████████████████| 6.8 MB 32.1 MB/s 
     |████████████████████████████████| 981 kB 34.9 MB/s 
     |████████████████████████████████| 263 kB 47.1 MB/s 
     |████████████████████████████████| 1.9 MB 25.7 MB/s 
     |████████████████████████████████| 1.2 MB 36.8 MB/s 
     |████████████████████████████████| 468 kB 45.9 MB/s 
     |████████████████████████████████| 3.3 MB 27.3 MB/s 
     |████████████████████████████████| 895 kB 40.1 MB/s 
  Created wheel for ktrain: filename=ktrain-0.27.3-py3-none-any.whl size=25283131 sha256=a449113e307b089484396f2828f5736753a53c5b3efc85a1989dbc696509016e
  Stored in directory: /root/.cache/pip/wheels/3c/ef/e1/7da805d8a5944e8a3ac0553831d832c00b1800b848939849cb
  Created wheel for seqeval: filename=seqeval-0.0.19-py3-none-any.whl size=9929 sha256=b6a5bdf4b94173bbe143369225840d5173b43607ba91ff378ffcd9c643f5e421
  Stored in directory: /root/.cache/pip/wheels/f5/ac/f1/4e13d7aff

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import ktrain
from ktrain import text

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/PreparedXethon20k.csv')

In [ ]:
df.head()

,Unnamed: 0,reviews,sentiment
0,6826,the last of the sequelsnot counting abbott and...,positive
1,1652,on the way home from a day at the beach four y...,negative
2,15452,will not seem to charge all the way and this i...,negative
3,1718,this was honestly the worst movie i have ever ...,negative
4,6940,and thats why historicbiographic movies are so...,positive


In [ ]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
df = df.dropna()

In [ ]:
df.head()

,reviews,sentiment
0,the last of the sequelsnot counting abbott and...,positive
1,on the way home from a day at the beach four y...,negative
2,will not seem to charge all the way and this i...,negative
3,this was honestly the worst movie i have ever ...,negative
4,and thats why historicbiographic movies are so...,positive


In [ ]:
df.shape

(20823, 2)

In [ ]:
X = list(df['reviews'])
y_ = list(df['sentiment'])

In [ ]:
# Encoding to 0's and 1's
y = [1 if labels == 'positive' else 0 for labels in df['sentiment']]

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =.2, random_state=0)

In [ ]:
print(len(X_train))

16658


In [ ]:
train_df = pd.concat([X_train, y_train], axis=0)
train_df.head()

TypeError: ignored

In [ ]:
# splitting the entire dataset to train and test splits
df_train = df.head(16000)
df_test = df.tail(4823)

In [ ]:
df_train.head()

,reviews,sentiment
0,the last of the sequelsnot counting abbott and...,positive
1,on the way home from a day at the beach four y...,negative
2,will not seem to charge all the way and this i...,negative
3,this was honestly the worst movie i have ever ...,negative
4,and thats why historicbiographic movies are so...,positive


In [ ]:
(X_train, y_train), (X_test, y_test), preprocess =  text.texts_from_df(
    train_df = df_train,
    text_column = 'reviews',
    label_columns = 'sentiment',
    val_df = df_test,
    maxlen = 200,
    preprocess_mode = 'bert'
)

['negative', 'positive']
   negative  positive
0       0.0       1.0
1       1.0       0.0
2       1.0       0.0
3       1.0       0.0
4       0.0       1.0
['negative', 'positive']
       negative  positive
16001       1.0       0.0
16002       1.0       0.0
16003       1.0       0.0
16004       0.0       1.0
16005       0.0       1.0
downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [ ]:
X_train[0].shape

(16000, 200)

In [ ]:
model = text.text_classifier(name='bert',
                             train_data = (X_train, y_train),
                             preproc = preprocess
                             )

Is Multi-Label? False
maxlen is 200
done.


In [ ]:
# Get the learning rate
learner = ktrain.get_learner(
    model = model,
    train_data = (X_train, y_train),
    val_data = (X_test, y_test),
    batch_size=4
)

In [ ]:
learner.fit_onecycle(lr=2e-5, epochs=1)



begin training using onecycle policy with max lr of 2e-05...
4000/4000 [==============================] - 1876s 469ms/step - loss: 0.3418 - accuracy: 0.8556 - val_loss: 0.2460 - val_accuracy: 0.9013


In [ ]:
learner

In [ ]:
predictor = ktrain.get_predictor(learner.model, preprocess)

In [ ]:
data = ['This movie was horrible',
        'this product is good to use and I will also use this in future',
        'I hate this product']

In [ ]:
predictor.predict(data)

['negative', 'positive', 'negative']

In [ ]:
predictor.save('bert_model')

/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [ ]:
from sklearn.metrics import accuracy_score,classification_report, confusion_matrix

In [ ]:
y_pred = predictor.predict(X_test)

ValueError: ignored